In [1]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [3]:
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY')

In [5]:
openai = OpenAI()
claude = anthropic.Anthropic()
qwencoder= OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

QWENCODER_MODEL = 'ticlazau/qwen2.5-coder-7b-instruct'
OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-haiku-20240307"


In [6]:
sys_msg="""
You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. 
Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. 
The C++ response needs to produce an identical output in the fastest possible time.
"""

In [7]:
def user_prompt_for( python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [20]:
def message_for(python):
    messages= [
        {'role': 'system', 'content': sys_msg},
        {'role':'user', 'content': user_prompt_for(python)}
    ]
    return messages

In [9]:
def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [16]:
def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=message_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [17]:
def optimize_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],)
    reply=''
    with result as stream:
        for chunk in result.text_stream:
            reply += chunk
            print(chunk, end='', flush=True)
    write_output(reply)

In [18]:
def optimize_qwen(python):
    stream = qwencoder.chat.completions.create(model = QWENCODER_MODEL, messages= message_for(python), stream=True)
    reply=''
    for chunk in stream:
        fragment= chunk.choices[0].delta.content or ''
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)
    

In [13]:
pycode = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [14]:
exec(pycode)

Result: 3.141592658589
Execution Time: 17.280430 seconds


In [21]:
optimize_gpt(pycode)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(long long iterations, double param1, double param2) {
    double result = 1.0;
    for (long long i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    
    double result = calculate(100'000'000, 4.0, 1.0) * 4.0;

    auto end_time = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> execution_time = end_time - start_time;

    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << execution_time.count() << " seconds" << std::endl;

    return 0;
}
```

In [23]:
exec(pycode)

Result: 3.141592658589
Execution Time: 17.876939 seconds
